In [1]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import time
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from mta_metrics import bert_sim, \
    bert_contained_in, chrp_contained_in
from mta_metrics.alignment import awesome_align, usw_nm
from translate import gpt, google
tqdm.pandas()

/Users/chany/miniforge3/envs/py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/Users/chany/research/mt-ambiguity/4_evaluation/data/base_1idiomPerRow.csv')
df.columns

Index(['idiom', 'meaning', 's_f_brkt', 's_l_brkt', 's_f', 's_l', 's_a'], dtype='object')

In [3]:
langs = {
    'de': 'German',
    'he': 'Hebrew', 
    'hi': 'Hindi',
    'hu': 'Hungarian', 
    'ko': 'Korean',
    'sp': 'Spanish',
    'yo': 'Yoruba',
    'zh': 'Chinese'
}

# Translate

### GPT-4

In [35]:
def gpt_wrapper(x, lang, time_passed=1):
    time.sleep(time_passed)
    try:
        return gpt(x, lang)
    except:
        if time_passed >= 8:
            print("backoff: ", time_passed)
        return gpt_wrapper(x, lang, time_passed * 2)
indices = [0, 128, 256, 384, 512]

In [96]:
lang = 'hu'
afl = 'f'

In [97]:
subset = 0
s_0 = df[f"s_{afl}"][indices[subset]: indices[subset + 1]].progress_apply(
    lambda x: gpt_wrapper(x, langs[lang]))
s_0

100%|██████████| 128/128 [04:01<00:00,  1.89s/it]


0      Annak ellenére, hogy intelligens volt, a halog...
1      Évekig keresték, de végül megtalálták az igazi...
2      Gyorsan a Török Ifjak közé került a cégnél az ...
3      Úgy döntött, elfogadja az állásajánlatot, mert...
4      Amikor az új alkalmazott problémákat kezdett o...
                             ...                        
123    Bocsánatot kértek, majd úgy döntöttek, hogy ti...
124    A csapat tiszta lappal végzett, az összes mérk...
125    Az érvelés után tisztáztuk a helyzetet vele az...
126    Felkészültünk az eseményre azzal, hogy befejez...
127    Közel hívta a vész, amikor szűkön megúszta az ...
Name: s_f, Length: 128, dtype: object

In [98]:
subset = 1
s_1 = df[f"s_{afl}"][indices[subset]: indices[subset + 1]].progress_apply(
    lambda x: gpt_wrapper(x, langs[lang]))
s_1

100%|██████████| 128/128 [04:03<00:00,  1.91s/it]


128    A projekt sikerének ellenére felhők jelentek m...
129    Miután az őr elment, tudtuk, hogy nincs akadál...
130    Sikeres üzleti vállalkozásuknak köszönhetően h...
131    Hidegvérrel találta magát a nagy prezentáció e...
132    Ő abbahagyta a dohányzást a hideg pulyka módsz...
                             ...                        
251    Belekerültek egy kellemetlen helyzetbe, amikor...
252    Az első munkanapján egy kihívást jelentő proje...
253    A cégnek azt vádolták, hogy összejátszik a ver...
254    Azért, hogy elkerüljük a félreértéseket, úgy d...
255    Egy drága hibát követően mély vízbe került a f...
Name: s_f, Length: 128, dtype: object

In [99]:
subset = 2
s_2 = df[f"s_{afl}"][indices[subset]: indices[subset + 1]].progress_apply(
    lambda x: gpt_wrapper(x, langs[lang]))
s_2

100%|██████████| 128/128 [04:06<00:00,  1.93s/it]


256    Amikor megérkeztünk, a buli teljes gőzzel zajl...
257    Azután kellett magyarázkodnia, hogy elfelejtet...
258    Ő egy természetes sportoló, benne van az ő vér...
259    Hónapokig tartó küzdelem után, vállalkozásunk ...
260    Mint kiszivárogtató, ő maga is a vállalat megt...
                             ...                        
379    Mindig valahogy furcsának tűntek, mintha patká...
380    Az ő pozitív hozzáállása egy napsugár volt a s...
381    Miután 12 órán keresztül dolgozott folyamatosa...
382    Elmentek nyaralni, hogy feltöltődjenek és viss...
383    A nyomozó egy hamis nyomra bukkant a gyanúsíto...
Name: s_f, Length: 128, dtype: object

In [100]:
subset = 3
s_3 = df[f"s_{afl}"][indices[subset]: indices[subset + 1]].progress_apply(
    lambda x: gpt_wrapper(x, langs[lang]))
s_3

100%|██████████| 128/128 [03:55<00:00,  1.84s/it]


384    Az régi étterem említése úgy tűnt, hogy ismerő...
385    A plagizált könyvnek úgy tűnt, mintha letépte ...
386    Ez egy új nap, kelj fel és ragyogj, és használ...
387    A cég politikájának megkérdőjelezésével kissé ...
388    Ebben a kisvárosban mélyen gyökereznek és mind...
                             ...                        
507    Ő sosem áll ki magáért és mindig sárga vonal v...
508    Ő otthon akart maradni és elmenni a buliba, de...
509    Nem lehet egyszerre megtartani egy jól fizető ...
510    Ha rosszul bánsz az emberekkel, akkor amikor n...
511    Amikor elkezdett beszélni a titokról, azt mond...
Name: s_f, Length: 128, dtype: object

In [101]:
df[f'p_{afl}_{lang}_gpt'] = pd.concat([s_0, s_1, s_2, s_3]).reset_index(drop=True)
df.head()

,idiom,meaning,s_f_brkt,s_l_brkt,s_f,s_l,s_a,p_a_de_gpt,p_f_de_gpt,p_l_de_gpt,p_a_he_gpt,p_f_he_gpt,p_l_he_gpt,p_f_hi_gpt,p_l_hi_gpt,p_a_hu_gpt,p_f_hu_gpt
0,Achilles heel,a vulnerable spot or weakness,"Despite his intelligence, procrastination was ...","During the anatomy class, we studied [[his Ach...","Despite his intelligence, procrastination was ...","During the anatomy class, we studied his Achil...",his Achilles heel,seine Achillesferse,Trotz seiner Intelligenz war Prokrastination s...,Während des Anatomieunterrichts haben wir sein...,עקב אכילס שלו,"למרות חכמתו, הדחייה הייתה העקב האחורי שלו.","במהלך השיעור של האנטומיה, למדנו על עקבו שלו ופ...","उसकी बुद्धिमता के बावजूद, टाल-मटोल उसकी असमर्थ...","एनाटॉमी क्लास के दौरान, हमने उसके एकिलीज हील औ...",az Achilles-sarka,"Annak ellenére, hogy intelligens volt, a halog..."
1,"Mr Right, Ms Right",the best future life partner for someone,"After years of searching, they finally [[found...","At the costume party, they [[found their Mr Ri...","After years of searching, they finally found t...","At the costume party, they found their Mr Righ...",found their Mr Right and Ms Right,haben ihren Mr. Right und ihre Ms. Right gefun...,Nach Jahren der Suche fanden sie endlich ihren...,Auf der Kostümparty fanden sie ihre Mr. Right ...,מצאו את הגבר הנכון והאישה הנכונה שלהם,"לאחר שנים של חיפושים, הם מצאו את הגבר הנכון וה...","במסיבת התחפושות, הם מצאו את התלבושות של מר וגב...","खोज के कई वर्षों के बाद, वे अंततः अपने मिस्टर ...","कॉस्ट्यूम पार्टी पर, उन्होंने दुकान में अपने म...",Megtalálták az Igazit (férfi és női változatban).,"Évekig keresték, de végül megtalálták az igazi..."
2,Young Turk,a young person who has many new ideas and want...,He quickly became [[a Young Turk in the compan...,The new employee was [[a Young Turk in the com...,He quickly became a Young Turk in the company ...,The new employee was a Young Turk in the compa...,a Young Turk in the company,ein Jungtürke in der Firma,Er wurde aufgrund seiner innovativen Ideen sch...,Der neue Mitarbeiter war ein Jungtürke im Unte...,טורק צעיר בחברה,הוא הפך מהר מאוד לטורק צעיר בחברה עקב רעיונותי...,"העובד החדש היה טורק צעיר בחברה, שעבר לאחרונה מ...",उसके नवाचारी विचारों के कारण वह जल्दी से कंपनी...,"नया कर्मचारी कंपनी में एक यंग तुर्क था, हाल ही...",egy fiatal török a társaságban,Gyorsan a Török Ifjak közé került a cégnél az ...
3,a bird in the hand,a possession that is worthy,"He decided to accept the job offer, believing ...","Holding [[a bird in the hand]], she carefully ...","He decided to accept the job offer, believing ...","Holding a bird in the hand, she carefully exam...",a bird in the hand,Ein Vogel in der Hand.,"Er entschied sich, das Jobangebot anzunehmen, ...","Indem sie einen Vogel in der Hand hielt, unter...",ציפור ביד,"הוא החליט לקבל את ההצעת עבודה, מאמין שציפור בי...","תופסת ציפור ביד, היא בדקה בזהירות את הנוצות שלה.",उसने नौकरी की पेशकश को स्वीकार करने का फैसला क...,"उसने हाथ में एक पक्षी पकड़ा हुआ था, उसने ध्यान...",egy madár a kézben,"Úgy döntött, elfogadja az állásajánlatot, mert..."
4,a cuckoo in the nest,an unwelcome intruder in a place or situation.,When the new employee started causing problems...,"While observing the bird's nest, the ornitholo...",When the new employee started causing problems...,"While observing the bird's nest, the ornitholo...",found a cuckoo in the nest,Ich habe einen Kuckuck im Nest gefunden.,"Als der neue Mitarbeiter anfing, Probleme zu v...","Während er das Vogelnest beobachtete, fand der...",מצאתי קוקו בקן.,"כאשר העובד החדש התחיל לגרום לבעיות, הם הבינו ש...","בזמן שהאורניתולוג צפה בקנים של הציפור, הוא מצא...","जब नया कर्मचारी समस्याएं उत्पन्न करने लगा, तब ...","पक्षी के घोंसले की निगरानी करते हुए, पक्षिविद ...",talált egy kakukkot a fészekben,Amikor az új alkalmazott problémákat kezdett o...


In [102]:
df.to_csv(f'ckpt/{afl}_{lang}.csv', index=False)

### Google

In [ ]:
def google_wrapper(x, lang):
    try:
        return google(x, lang)
    except:
        time.sleep(5)
        return google(x, lang)

In [ ]:
for lang in langs:
    print(f"Begin Google Translation: {lang}")
    df[f'p_f_{lang}_google'] = google_wrapper(df['s_f'], lang)
    df[f'p_l_{lang}_google'] = google_wrapper(df['s_l'], lang)
    df[f'p_a_{lang}_google'] = google_wrapper(df['s_a'], lang)
    print()

## Sample Translation: Spanish

## USW, NM

In [ ]:
alignments_a = awesome_align(df['s_a'], df['p_a'])
alignments_f = awesome_align(df['s_f'], df['p_f'])
alignments_l = awesome_align(df['s_l'], df['p_l'])
df['usw_a'], df['nm_a'], usw_a, nm_a = usw_nm(df['s_a'], alignments_a, df['p_a'])
df['usw_f'], df['nm_f'], usw_f, nm_f = usw_nm(df['s_f'], alignments_f, df['p_f'])
df['usw_l'], df['nm_l'], usw_l, nm_l = usw_nm(df['s_l'], alignments_l, df['p_l'])

In [ ]:
print(usw_a, usw_f, usw_l)
print(nm_a, nm_f, nm_l)

## Contained In

In [ ]:
df['contained_in(p_a,p_f)_BERT'] = bert_score_contained_in(df['p_a'], df['p_f'], 'sp')
df['contained_in(p_a,p_l)_BERT'] = bert_score_contained_in(df['p_a'], df['p_l'], 'sp')
df['contained_in(p_a,p_f)_chrP'] = chrp_contained_in(df['p_a'], df['p_f'])
df['contained_in(p_a,p_l)_chrP'] = chrp_contained_in(df['p_a'], df['p_l'])
df['sensitivity_BERT'] = abs(df['contained_in(p_a,p_l)_BERT'] - df['contained_in(p_a,p_f)_BERT'])
df['sensitivity_chrP'] = abs(df['contained_in(p_a,p_l)_chrP'] -  df['contained_in(p_a,p_f)_chrP'])

In [ ]:
df